In [16]:
torch.cuda.is_available()
#torch.cuda.get_device_name(0)

True

In [1]:
from utils import *
from IPython.display import display, HTML
from fastai2.text.all import *

In [2]:
path = untar_data(URLs.IMDB)
path.ls()

(#7) [Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/README'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/imdb.vocab'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/unsup'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/train'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/tmp_lm'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/tmp_clas'),Path('/home/jayajasodani_gmail_com/.fastai/data/imdb/test')]

In [3]:
files = get_text_files(path, folders=['train','test','unsup'])

In [4]:
txt = files[0].open().read()
txt[:75]

"Am not from America, I usually watch this show on AXN channel, I don't know"

In [5]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks,30))

(#215) ['Am','not','from','America',',','I','usually','watch','this','show','on','AXN','channel',',','I','do',"n't",'know','why','this','respected','channel','air','such','sucking','program','in','prime','time','slot'...]


In [6]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [6]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt),30))

(#240) ['xxbos','xxmaj','am','not','from','xxmaj','america',',','i','usually','watch','this','show','on','xxup','axn','channel',',','i','do',"n't",'know','why','this','respected','channel','air','such','sucking','program'...]


In [8]:
#defaults.text_proc_rules
defaults

namespace(cpus=8,
          use_cuda=None,
          activation=torch.nn.modules.activation.ReLU,
          callbacks=[fastai2.callback.core.TrainEvalCallback,
                     fastai2.learner.Recorder,
                     fastai2.callback.progress.ProgressCallback],
          lr=0.001,
          text_spec_tok=['xxunk',
                         'xxpad',
                         'xxbos',
                         'xxeos',
                         'xxfld',
                         'xxrep',
                         'xxwrep',
                         'xxup',
                         'xxmaj'],
          text_proc_rules=[<function fastai2.text.core.fix_html(x)>,
                           <function fastai2.text.core.replace_rep(t)>,
                           <function fastai2.text.core.replace_wrep(t)>,
                           <function fastai2.text.core.spec_add_spaces(t)>,
                           <function fastai2.text.core.rm_useless_spaces(t)>,
                           <func

In [10]:
??replace_space

In [11]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX')) #"String repr of up to `max_n` items of (possibly lazy) collection `c`"

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup'...]"

In [7]:
txts = L(o.open().read() for o in files[:2000])

In [10]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txt)
    return ' '.join(first(sp([txt]))[:40])

In [11]:
subword(1000)

'▁ A m ▁ n o t ▁ f r o m ▁ A m e r i c a , ▁ I ▁ u s u a l l y ▁ w a t c h ▁ t h'

In [12]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#240) ['xxbos','xxmaj','am','not','from','xxmaj','america',',','i','usually','watch','this','show','on','xxup','axn','channel',',','i','do',"n't",'know','why','this','respected','channel','air','such','sucking','program','in'...]


In [22]:
toks200 = txts[:200].map(tkn) #Read from first 200 files and tokenize
toks200[0]               #This becomes 'toks' due to some reason

(#240) ['xxbos','xxmaj','am','not','from','xxmaj','america',',','i','usually'...]

In [26]:
num=Numericalize() 
num.setup(toks200)           #Give numbers to words in 200 files 
coll_repr(num.vocab,20)      #Vocab sorts words in DESCENDING order frequency (with special words first) 

"(#2072) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','it','i'...]"

In [27]:
nums=num(toks)[:20]; nums

tensor([   0,   34,   51,    0,   11,    0,  818,  117,   20,  116,   32,    0, 1519,   11,    0,   65,   43,  114,  135,   20])

In [16]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj am not from xxmaj america , i usually watch this show on xxup xxunk channel , i do'

In [17]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len=6,15
d_tokens=np.array([tokens[i*seq_len:(i+1)*seq_len]for i in range(bs)])
df=pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [18]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [19]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [25]:
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [28]:
nums200 = toks200.map(num)
toks200; nums200

(#200) [TensorText([   2,    8,  233,   34,   51,    8,  716,   11,   19,  818,  117,   20,  116,   32,    7,    0, 1519,   11,   19,   65,   43,  114,  135,   20,    0, 1519,  951,  146,    0,  819,   17,    0,
          76,    0,   10,    8,    0,   14,    8,  367,   23,    8,  204,    8,  647,    8, 1173,    8,    0,   11,   20,   76,   37,   16, 1174,   12,  263,    0,   14, 1520,   17,    9,  388,  264,
          10,    8,   17,  286,  421,   12,  717,   14,    0,  234,  118,  648,  578, 1521,   51,   32,  516,   15,  199,   27,   12,  101, 1522,   14,  204,  156,  143,    9,  340,    0, 1523,   72,
        1521,   10,   19,   65,   43,  114,   48,    9,  480,  952,   20,  422,  341,   27,    9,  116,   10,    8,   18,   56,  193,   15,  117,   27,   11,   17,   47, 1175,   84,   96,  110, 1524,
          52,    0,   11,   64,  217,    0,   11, 1525,   11,    0,   11,    0,  340,  579,  186,    0,   18,   23,    0,   15,  238,  481,   13,  953,   10,    8,    9,   80,  954,  166, 

In [21]:
dl=LMDataLoader(nums200)
first(dl)

(LMTensorText([[   2,    8,  233,  ...,   12,  717,   14],
         [ 149,   39,   17,  ...,    8,  263,    8],
         [ 175,   15,    9,  ...,    8,  534,    8],
         ...,
         [ 582,   17,    0,  ...,   14, 1676,   10],
         [   8,  626,   22,  ..., 1154,   27,  123],
         [1652,   23,    0,  ...,    7,  857,    7]]),
 tensor([[   8,  233,   34,  ...,  717,   14,    0],
         [  39,   17,   58,  ...,  263,    8,  368],
         [  15,    9,  735,  ...,  534,    8,  837],
         ...,
         [  17,    0,  594,  ..., 1676,   10,    8],
         [ 626,   22,    0,  ...,   27,  123,  118],
         [  23,    0,   27,  ...,  857,    7,    0]]))

In [28]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [29]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj am not from xxmaj america , i usually watch this show on xxup xxunk channel , i do'

In [30]:
' '.join(num.vocab[o] for o in y[0][:20])

"xxmaj am not from xxmaj america , i usually watch this show on xxup xxunk channel , i do n't"

In [8]:
get_imdb = partial(get_text_files, folders=['train','test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items = get_imdb, splitter = RandomSplitter(0.1)
).dataloaders(path,path=path, bs=64, seq_len=80)

In [32]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj it 's not too often that i really want to see a particular movie . xxmaj but this was one of those exceptions . xxmaj the trailers were pretty funny , and i figured if they were just a small segment of what the rest of the movie is like … it should be hilarious . xxmaj well , i think they used the funniest parts of the movie to put in those trailers . xxmaj it seems","xxmaj it 's not too often that i really want to see a particular movie . xxmaj but this was one of those exceptions . xxmaj the trailers were pretty funny , and i figured if they were just a small segment of what the rest of the movie is like … it should be hilarious . xxmaj well , i think they used the funniest parts of the movie to put in those trailers . xxmaj it seems like"
1,"courier and helped himself to a suitcase with $ 400 , xxrep 3 0 . xxmaj and it seems that xxmaj bain 's missing beau is the prime suspect . xxmaj the mob is not amused and has a contract out on xxmaj gianni . xxmaj boone and xxmaj dunn find that double cross on top of double cross is the order of the day with this bunch . xxmaj buono is short of cash due to some large gambling","and helped himself to a suitcase with $ 400 , xxrep 3 0 . xxmaj and it seems that xxmaj bain 's missing beau is the prime suspect . xxmaj the mob is not amused and has a contract out on xxmaj gianni . xxmaj boone and xxmaj dunn find that double cross on top of double cross is the order of the day with this bunch . xxmaj buono is short of cash due to some large gambling losses"


In [9]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

In [10]:
learn.fit_one_cycle(1,2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.136819,3.929943,0.298000,50.904068,2:34:34


In [11]:
learn.save('1epoch')

In [12]:
learn = learn.load('1epoch')

In [13]:
learn.unfreeze()
learn.fit_one_cycle(10,2e-3)

KeyboardInterrupt: 

In [15]:
learn.save_encoder('finetuned')

In [17]:
#Text Generation

In [18]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [19]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,neg


In [29]:
nums_samp = toks200[:10].map(num)

In [30]:
nums_samp.map(len)

(#10) [240,60,144,149,101,102,597,125,161,358]

In [31]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [32]:
learn = learn.load_encoder('finetuned')

In [33]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.393038,0.247902,0.899640,08:42


In [34]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.271667,0.210066,0.918520,09:58


In [35]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.213087,0.181578,0.931880,13:52


In [36]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

KeyboardInterrupt: 